In [4]:
from google.colab import files
import pandas as pd
uploaded = files.upload()



Saving tweets.csv to tweets.csv


In [5]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('tweets.csv')
df.head()

,id,keyword,location,text,target
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1
1,1,ablaze,NaN,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1
2,2,ablaze,New York City,Arsonist sets cars ablaze at dealership https://t.co/gOQvyJbpVI,1
3,3,ablaze,"Morgantown, WV",Arsonist sets cars ablaze at dealership https://t.co/0gL7NUCPlb https://t.co/u1CcBhOWh9,1
4,4,ablaze,NaN,"""Lord Jesus, your love brings freedom and pardon. Fill me with your Holy Spirit and set my heart ablaze with your l… https://t.co/VlTznnPNi8",0


In [6]:
df['Description_Clean'] = df['text'].str.lower()
df.head(2)

,id,keyword,location,text,target,Description_Clean
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1,"communal violence in bhainsa, telangana. ""stones were pelted on muslims' houses and some houses and vehicles were set ablaze…"
1,1,ablaze,NaN,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1,"telangana: section 144 has been imposed in bhainsa from january 13 to 15, after clash erupted between two groups on january 12. po…"


In [7]:
df['Description_Clean'] = df['Description_Clean'].str.replace('\xa0', ' ')
df.head(2)

,id,keyword,location,text,target,Description_Clean
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1,"communal violence in bhainsa, telangana. ""stones were pelted on muslims' houses and some houses and vehicles were set ablaze…"
1,1,ablaze,NaN,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1,"telangana: section 144 has been imposed in bhainsa from january 13 to 15, after clash erupted between two groups on january 12. po…"


In [8]:
df.text.iloc[0]

'Communal violence in Bhainsa, Telangana. "Stones were pelted on Muslims\' houses and some houses and vehicles were set ablaze…'

In [9]:
df.Description_Clean.iloc[0]

'communal violence in bhainsa, telangana. "stones were pelted on muslims\' houses and some houses and vehicles were set ablaze…'

In [10]:
df['Description_Clean'] = df['Description_Clean'].str.replace(r'[^\w\s]', '', regex=True)
df.head(2)

,id,keyword,location,text,target,Description_Clean
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1,communal violence in bhainsa telangana stones were pelted on muslims houses and some houses and vehicles were set ablaze
1,1,ablaze,NaN,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1,telangana section 144 has been imposed in bhainsa from january 13 to 15 after clash erupted between two groups on january 12 po


In [11]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [12]:
desc = df.Description_Clean.loc[0]
desc

'communal violence in bhainsa telangana stones were pelted on muslims houses and some houses and vehicles were set ablaze'

In [14]:
doc = nlp(desc)
doc

communal violence in bhainsa telangana stones were pelted on muslims houses and some houses and vehicles were set ablaze

In [15]:
[token.lemma_ for token in doc if not token.is_stop][:10]

['communal',
 'violence',
 'bhainsa',
 'telangana',
 'stones',
 'pelt',
 'muslim',
 'house',
 'house',
 'vehicle']

In [16]:
' '.join([token.lemma_ for token in doc if not token.is_stop])

'communal violence bhainsa telangana stones pelt muslim house house vehicle set ablaze'

In [17]:
def token_lemma_nonstop(text):
    doc = nlp(text)
    output = [token.lemma_ for token in doc if not token.is_stop]
    output = ' '.join(output)
    return output

In [18]:
df['Description_Clean'] = df.Description_Clean.apply(token_lemma_nonstop)
df.head(2)

,id,keyword,location,text,target,Description_Clean
0,0,ablaze,NaN,"Communal violence in Bhainsa, Telangana. ""Stones were pelted on Muslims' houses and some houses and vehicles were set ablaze…",1,communal violence bhainsa telangana stones pelt muslim house house vehicle set ablaze
1,1,ablaze,NaN,"Telangana: Section 144 has been imposed in Bhainsa from January 13 to 15, after clash erupted between two groups on January 12. Po…",1,telangana section 144 impose bhainsa january 13 15 clash erupt group january 12 po


In [21]:
print(df.columns)


Index(['id', 'keyword', 'location', 'text', 'target', 'Description_Clean'], dtype='object')


In [24]:
import multiprocessing as mp
from collections import Counter
import pandas as pd
def process_chunk(chunk):
    word_counts = Counter()
    for text in chunk:
        tokens = text.split()
        word_counts.update(tokens)
    return word_counts
def merge_counters(counters):
    total = Counter()
    for c in counters:
        total.update(c)
    return total
if __name__ == "__main__":
    texts = df['Description_Clean'].dropna().tolist()
    num_processes = mp.cpu_count()
    chunk_size = len(texts) // num_processes
    chunks = [texts[i:i + chunk_size] for i in range(0, len(texts), chunk_size)]
    with mp.Pool(processes=num_processes) as pool:
        results = pool.map(process_chunk, chunks)
    final_counts = merge_counters(results)
    word_freq_df = pd.DataFrame(final_counts.items(), columns=["Word", "Frequency"]).sort_values(by="Frequency", ascending=False)
    print("Top 20 Most Frequent Words:\n")
    print(word_freq_df.head(20).to_string(index=False))


Top 20 Most Frequent Words:

  Word  Frequency
   not        975
     m        589
     s        569
   amp        567
  like        554
people        538
  fire        491
  year        340
  need        312
    go        307
   say        304
  know        297
  time        296
 think        286
   new        276
   get        274
   day        264
  come        252
  want        249
     t        237
